# **Лабораторная работа №1 по вычислительной математикe.**

## Работу выполнил: Аль Мажариш Гасем, Б01 - 202а.


In [115]:
import numpy as np

## Формулировка задачи

II.10.2 Сделать по 5 итераций методов Якоби, Зейделя и градиентного спуска для системы:

$4 x_1 + x_2 = 15$

$x_1 + 4 x_2 + x_3 = 15$

$3 x_2 + 3 x_3 = 10$

## Определение размера СЛАУ и допустимой погрешности.

In [116]:
SIZE = 3
TOL = 1e-5
MAX_ITERATIONS = 5

## Определение функций расчета нормы $\|x\|_i$ вектора.

In [117]:
def cube_vec_norm(vec):
    return max(abs(vec))
    
def oct_vec_norm(vec):
    return abs(max(vec))

def evk_vec_norm(vec):
    return np.sqrt(sum(vec * vec))

vec_norm_list = [cube_vec_norm, oct_vec_norm, evk_vec_norm]

## Метод диагональной прогонки
Этот метод применяется к СЛАУ, где матрица коэффициентов является трехдиагональной.

In [118]:
def tridiagonal_arg(A):
    size = len(A[0])
    a = np.zeros(size - 1)
    b = np.zeros(size)
    c = np.zeros(size - 1)
    
    for i in range(size - 1):
        a[i] = A[i + 1][i]
        b[i] = A[i][i]
        c[i] = A[i][i + 1]

    b[size - 1] = A[size - 1][size - 1]

    return a, b, c
                

def tridiagonal_solver(a, b, c, d):
    n = len(d)
    P = np.zeros(n - 1)
    Q = np.zeros(n)
    
    # Задаём прогоночные коэффициенты
    P[0] = -c[0] / b[0]
    Q[0] = d[0] / b[0]

    # Прямой ход
    for i in range(1, n - 1):
        denominator = b[i] + a[i - 1] * P[i - 1]
        P[i] = -c[i] / denominator
        Q[i] = (d[i] - a[i - 1] * Q[i - 1]) / denominator
    
    Q[n - 1] = (d[n - 1] - a[n - 2] * Q[n - 2]) / (b[n - 1] + a[n - 2] * P[n - 2])

    # Обратный ход
    x = np.zeros(n)
    x[n - 1] = Q[n - 1]
    
    for i in range(n - 2, -1, -1):
        x[i] = P[i] * x[i + 1] + Q[i]
    
    return x

## Инициализация матрицы СЛАУ

In [119]:
A = np.array([[4, 1, 0], [1, 4, 1], [0, 1, 3]], dtype=float)
b = np.array([15, 15, 10], dtype=float)
x0 = np.zeros_like(b)

## Метод Якоби
Метод предполагает разбиение матрицы на LDU - Lower, Diagonale, Upper

In [120]:
def jacobi_method(A, b, x0, tol=TOL, max_iterations=MAX_ITERATIONS):
    # Инициализируем
    n = len(A)
    x = x0.copy()
    x_new = np.zeros_like(x)

    # Итерируем
    for it_count in range(max_iterations):
        for i in range(n):
            s = sum(A[i][j] * x[j] for j in range(n) if j != i)
            x_new[i] = (b[i] - s) / A[i][i]
        
        if evk_vec_norm(x_new - x) < tol:
            return x_new, it_count
        x = x_new.copy()

    return x_new, max_iterations

## Метод Зейделя
Метод Зейделя — это модификация метода Якоби, где на каждой итерации сразу используется обновленное значение.

In [121]:
def gauss_seidel_method(A, b, x0, tol=TOL, max_iterations=MAX_ITERATIONS):
    # Инициализируем
    n = len(A)
    x = x0

    # Итерируем
    for it_count in range(max_iterations):
        x_new = x.copy()
        for i in range(n):
            s1 = sum(A[i][j] * x_new[j] for j in range(i))
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i][i]
        
        if evk_vec_norm(x_new - x) < tol:
            return x_new, it_count
        x = x_new

    return x_new, max_iterations

## Вариационный метод
Вариационный метод решения СЛАУ основан на минимизации функционала, связанного с матрицей системы и правой частью.

In [125]:
def variational_method(A, b, x0, tol=TOL, max_iterations=MAX_ITERATIONS):
    x = x0
    r = b - A @ x
    p = r

    for it_count in range(max_iterations):
        alpha = (r @ r) / (p @ A @ p)    # Шаг спуска
        x_new = x + alpha * p            # Обновление решения
        r_new = r - alpha * A @ p        # Обновление невязки
        
        if evk_vec_norm(r_new) < tol:
            return x_new, it_count

        # Обновление направления поиска
        beta = (r_new @ r_new) / (r @ r)
        p = r_new + beta * p
        r = r_new
        x = x_new

    return x_new, max_iterations

In [126]:
x_jacobi, it_jacobi = jacobi_method(A, b, x0)
x_seidel, it_seidel = gauss_seidel_method(A, b, x0)
x_tridiagonal = tridiagonal_solver(*tridiagonal_arg(A), b)
x_variational, it_variational = variational_method(A, b, x0)

print("Решение методом прогонки:", x_tridiagonal)
print("Решение методом Якоби:", x_jacobi, "Кол-во итераций:", it_jacobi)
print("Решение методом Зейделя:", x_seidel, "Кол-во итераций:", it_seidel)
print("Решение вариационным методом:", x_variational, "Кол-во итераций:", it_variational)

Решение методом прогонки: [3.17073171 2.31707317 2.56097561]
Решение методом Якоби: [3.18305122 2.34754774 2.57740162] Кол-во итераций: 5
Решение методом Зейделя: [3.17034757 2.31729725 2.56090092] Кол-во итераций: 5
Решение вариационным методом: [3.17073171 2.31707317 2.56097561] Кол-во итераций: 2


## Выводы
Ответы, полученные разными методами, близки друг к другу, что свидетельствует их правильной реализации.
Наилучшую точность демонстрирует метод градиентного спуска. Также видно, что при огромной схожести метод Зейделя превосходит метод Якоби.